In [1]:
import numpy as np
import pandas as pd

In [2]:
user_activity = pd.read_csv('kaggle/user_activity.csv')
user_activity_test = pd.read_csv('kaggle/user_activity_test.csv')
structure = pd.read_csv('kaggle/structure.csv')
targets = pd.read_csv('kaggle/targets.csv').set_index('user_id')

In [3]:
user_activity.head()

,user_id,action,step_id,time,step_type,step_cost
0,5920,discovered,2681,1403165936,text,0
1,5920,passed,2681,1403165936,text,0
2,5920,viewed,2681,1403165936,text,0
3,5920,passed,2682,1403165946,text,0
4,5920,viewed,2682,1403165946,text,0


In [4]:
sum_cost = user_activity.groupby('user_id').sum().step_cost.values.reshape(-1, 1)
sum_cost_test = user_activity_test.groupby('user_id').sum().step_cost.values.reshape(-1, 1)

In [5]:
action_count = user_activity.groupby('user_id').count().action.values.reshape(-1, 1)
action_count_test = user_activity_test.groupby('user_id').count().action.values.reshape(-1, 1)

In [7]:
passed = user_activity.groupby('user_id').apply(lambda series: len(series[series.action == 'passed'])).values.reshape(-1, 1)
passed_test = user_activity_test.groupby('user_id').apply(lambda series: len(series[series.action == 'passed'])).values.reshape(-1, 1)

In [8]:
discovered = user_activity.groupby('user_id').apply(lambda series: len(series[series.action == 'discovered'])).values.reshape(-1, 1)
discovered_test = user_activity_test.groupby('user_id').apply(lambda series: len(series[series.action == 'discovered'])).values.reshape(-1, 1)

In [9]:
viewed = user_activity.groupby('user_id').apply(lambda series: len(series[series.action == 'viewed'])).values.reshape(-1, 1)
viewed_test = user_activity_test.groupby('user_id').apply(lambda series: len(series[series.action == 'viewed'])).values.reshape(-1, 1)

In [18]:
user_activity_test

user_id           action  step_id        time step_type  step_cost
0         20698           passed     7225  1410725206      text          0
1         20698           viewed     7225  1410725206      text          0
2         20698       discovered     7225  1410725206      text          0
3         20698           passed     7230  1410725214      text          0
4         20698       discovered     7230  1410725214      text          0
5         20698           viewed     7230  1410725214      text          0
6         20698           passed     7231  1410725235      text          0
7         20698       discovered     7231  1410725235      text          0
8         20698           viewed     7231  1410725235      text          0
9         20698       discovered     7232  1410725261      text          0
10        20698           passed     7232  1410725261      text          0
11        20698           viewed     7232  1410725261      text          0
12        20698       discovered     7233  1410725292      text          0
13        20698           viewed     7233  1410725292      text          0
14        20698           passed     7233  1410725293      text          0
15        20698           viewed     7225  1410725327      text          0
16        20698           viewed     7230  1410725329      text          0
17        20698           viewed     7231  1410725329      text          0
18        20698           viewed     7233  1410725330      text          0
19        20698           viewed     7232  1410725330      text          0
20        20698           viewed     2828  1410725340      text          0
21        20698       discovered     2828  1410725340      text          0
22        20698           passed     2828  1410725341      text          0
23        20698       discovered     4848  1410725367     video          0
24        20698           passed     4848  1410725367     video          0
25        20698           viewed     4848  1410725367     video          0
26        20698       discovered     4850  1410725407     video          0
27        20698           passed     4850  1410725407     video          0
28        20698           viewed     4850  1410725407     video          0
29        20698           viewed     4851  1410725494     video          0
...         ...              ...      ...         ...       ...        ...
539220        9           viewed     4851  1465167838     video          0
539221        9       discovered     6219  1465167851     video          0
539222        9           viewed     6219  1465167851     video          0
539223        9           passed     6219  1465167851     video          0
539224        9           passed     6220  1465167867     video          0
539225        9           viewed     6220  1465167867     video          0
539226        9       discovered     6220  1465167867     video          0
539227        9           passed     6221  1465167873     video          0
539228        9       discovered     6221  1465167873     video          0
539229        9           viewed     6221  1465167873     video          0
539230        9           passed     6222  1465167877     video          0
539231        9           viewed     6222  1465167877     video          0
539232        9       discovered     6222  1465167877     video          0
539233        9           viewed     2824  1465167916    choice          1
539234        9       discovered     2824  1465167916    choice          1
539235        9  started_attempt     2824  1465167919    choice          1
539236        9  started_attempt     2824  1465167951    choice          1
539237        9  started_attempt     2824  1465167983    choice          1
539238        9           passed     2824  1465167986    choice          1
539239        9           viewed     2825  1465167988    choice          1
539240        9       discovered     2825  1465167988    choice          1
539241        9  started_attempt 

In [44]:
def calc_mean_diff(series):
    diffs = []
    prev = series.values[0][3]
    for t in series.values:
        diffs.append(t[3] - prev)
        prev = t[3]
    return sum(diffs) / len(diffs)
mean_diff = user_activity.groupby('user_id').apply(calc_mean_diff).values.reshape((-1, 1))
mean_diff_test = user_activity_test.groupby('user_id').apply(calc_mean_diff).values.reshape((-1, 1))

In [74]:
def calc_max_diff(series):
    diffs = []
    prev = series.values[0][3]
    for t in series.values:
        diffs.append(t[3] - prev)
        prev = t[3]
    return max(diffs)
max_diff = user_activity.groupby('user_id').apply(calc_max_diff).values.reshape((-1, 1))
max_diff_test = user_activity_test.groupby('user_id').apply(calc_max_diff).values.reshape((-1, 1))

In [75]:
ind = np.array(user_activity.groupby('user_id').sum().index).reshape(-1)
features = sum_cost
features = np.concatenate((features, action_count), axis=1)
features = np.concatenate((features, passed), axis=1)
features = np.concatenate((features, discovered), axis=1)
features = np.concatenate((features, viewed), axis=1)
#features = np.concatenate((features, mean_diff), axis=1)
features = np.concatenate((features, max_diff), axis=1)
features

array([[   997,   1947,    254,    322,   1371,   9401],
       [     0,     29,      9,      9,     11,     53],
       [    50,    129,     31,     31,     46, 353011],
       ..., 
       [     0,     21,      7,      7,      7,     32],
       [     0,      4,      1,      1,      2,    211],
       [     0,      3,      1,      1,      1,      1]])

In [76]:
features_test = sum_cost_test
features_test = np.concatenate((features_test, action_count_test), axis=1)
features_test = np.concatenate((features_test, passed_test), axis=1)
features_test = np.concatenate((features_test, discovered_test), axis=1)
features_test = np.concatenate((features_test, viewed_test), axis=1)
#features_test = np.concatenate((features_test, max_diff_test), axis=1)
features_test

array([[     0,      3,      1,      1,      1,      0],
       [    11,     61,     18,     18,     20,     69],
       [     6,     17,      4,      5,      8, 385505],
       ..., 
       [     0,     15,      5,      5,      5,      6],
       [     0,      3,      1,      1,      1,      1],
       [     0,     21,      7,      7,      7,    195]])

In [77]:
true = targets.passed[ind]

In [78]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score

X_train, X_val, Y_train, Y_val = train_test_split(features, true, test_size=0.3)

In [81]:
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
model = SGDClassifier()

parameters = {
    'epsilon': np.linspace(0, 1, 10),
    'n_iter': np.arange(1, 50),
    'alpha': np.linspace(0.1, 1, 10),
    'average': [False],
    'power_t': [39],
    'penalty': ['none'],
    'loss': ['squared_hinge']
}

CV_rfc = GridSearchCV(estimator=model, param_grid=parameters, cv= 5)
CV_rfc.fit(X_train, Y_train)
CV_rfc.best_params_

{'alpha': 0.30000000000000004,
 'average': False,
 'epsilon': 0.0,
 'loss': 'squared_hinge',
 'n_iter': 1,
 'penalty': 'none',
 'power_t': 39}

In [84]:
model = SGDClassifier(**CV_rfc.best_params_)
model.fit(X_train, Y_train)

SGDClassifier(alpha=0.30000000000000004, average=False, class_weight=None,
       epsilon=0.0, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='squared_hinge', n_iter=1, n_jobs=1,
       penalty='none', power_t=39, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [85]:
f1_score(Y_val, model.predict(X_val))

0.015774180157741801

In [201]:
def create_submission(X, name):
    np.savetxt('%s.csv'%name, X, delimiter=',', fmt="%d", header='user_id,passed', comments='')

ind_test = user_activity_test.groupby('user_id').sum().index.values.reshape(-1, 1)
res = np.concatenate((ind_test, model.predict(features_test).reshape(-1, 1)), axis=1)

create_submission(res, 'sgdoptimized')

In [164]:
user_activity_test.groupby('user_id').count()

,action,step_id,time,step_type,step_cost
user_id,,,,,
5,3,3,3,3,3
9,61,61,61,61,61
21,17,17,17,17,17
26,45,45,45,45,45
29,1084,1084,1084,1084,1084
32,105,105,105,105,105
36,383,383,383,383,383
44,551,551,551,551,551
51,155,155,155,155,155
